In [ ]:
import psutil
import pyttsx3
import time

def set_voice_properties(engine):
    # Adjust voice properties for a more natural sound
    rate = engine.getProperty('rate')   # Getting the current voice rate
    engine.setProperty('rate', rate - 30)  # Decrease the rate to make it sound more natural
    volume = engine.getProperty('volume')   # Getting the current volume level
    engine.setProperty('volume', volume + 10.00)  # Increase the volume a bit

def get_battery_details():
    battery = psutil.sensors_battery()
    if battery:
        percent = battery.percent
        plugged = battery.power_plugged
        if plugged:
            status = "Your battery is plugged to a power source."
        else:
            status = "Your battery is not plugged to any power source. Please plug in the charger to avoid battery depletion."
        return f"Your Battery level is: {percent}% | Status: {status}"
    else:
        return "Your Battery status not available."

def welcome_message():
    # Voice out the welcome message along with battery details
    engine = pyttsx3.init()
    set_voice_properties(engine)
    message = "Welcome to the Battery Reminder Desktop App. "
    battery_details = get_battery_details()
    message += battery_details
    engine.say(message)
    engine.runAndWait()

def power_plugged_message():
    # Voice out the message when the power source is plugged in
    engine = pyttsx3.init()
    set_voice_properties(engine)
    message = "Your laptop is now plugged to a power source."
    engine.say(message)
    engine.runAndWait()

def power_unplugged_message():
    # Voice out the message when the power source is unplugged
    engine = pyttsx3.init()
    set_voice_properties(engine)
    message = "Your laptop is now running on battery power."
    engine.say(message)
    engine.runAndWait()

def display_battery_status(remind_charge, remind_unplug):
    prev_percent = None
    plugged = None
    welcome_message()  # Call the welcome message function
    while True:
        battery = psutil.sensors_battery()
        if battery:
            percent = battery.percent
            new_plugged = battery.power_plugged

            if plugged is None:
                # First iteration, do nothing
                pass
            elif new_plugged and not plugged:
                power_plugged_message()
            elif not new_plugged and plugged:
                power_unplugged_message()
            
            plugged = new_plugged
            time.sleep(60)  # Check battery level every 60 seconds
            if plugged and percent >= remind_charge:
                message = f"Your battery is currently fully charged. Please unplug the charger to save battery."
                print(message)
                # Voice out the reminder
                engine = pyttsx3.init()
                set_voice_properties(engine)
                engine.say(message)
                engine.runAndWait()
            elif not plugged and prev_percent is not None and percent < 70 and prev_percent - percent >= 1:
                message = f"Your battery level dropped to {percent}%. Please plug in the charger to avoid battery depletion."
                print(message)
                # Voice out the reminder
                engine = pyttsx3.init()
                set_voice_properties(engine)
                engine.say(message)
                engine.runAndWait()
            elif plugged and percent > 90:
                message = f"Your battery level is above 90%. Consider unplugging the charger to extend battery life."
            else:
                message = None
            prev_percent = percent

        

if __name__ == "__main__":
    # Set the battery percent thresholds for reminders
    remind_charge = int(input("Set battery percent to remind you to unplug (e.g., 99): "))
    remind_unplug = int(input("Set battery percent to remind you to plug in (e.g., 70): "))
    
    display_battery_status(remind_charge, remind_unplug)

Set battery percent to remind you to unplug (e.g., 99): 99
Set battery percent to remind you to plug in (e.g., 70): 70
